1. Working with IP Addresses 

a) Classful Addressing 

b) Classless Inter-Domain Routing (CIDR) 

Simple tool to manipulate and analyse IP address

In [ ]:
import ipaddress

def analyse_ip(ip_str):
    # Create an IP interface object
    ip = ipaddress.ip_interface(ip_str)

    print(f"Address: {ip.ip}")
    print(f"Network: {ip.network}")
    print(f"Netmask: {ip.netmask}")
    print(f"Is private: {ip.ip.is_private}")
    print(f"Is global: {ip.ip.is_global}")
    #Exercise 1:  Extend the script to calculate: 
    print(f"Broadcast address: {ip.network.broadcast_address}")
    print(f"First host: {ip.network.network_address +1}")
    print(f"Last host: {ip.network.network_address -1}")
    print(f"Total usable hosts {(2**(32 -30)) - 2}") 
    print(f"CIDR prefix ip: {ip.with_prefixlen}")

    # List all hosts in the network
    if ip.network.num_addresses < 256: # Only for small networks
        print("\nHosts in network:")
        for host in ip.network.hosts():
            print(host) 
            
# Example usage
print(analyse_ip('192.168.1.1'))
print(analyse_ip('192.168.1.1/24'))


Address: 192.168.1.1
Network: 192.168.1.0/30
Netmask: 255.255.255.252
Is private: True
Is global: False
Broadcast address: 192.168.1.3
first network: 192.168.1.1
last network: 192.168.0.255
total of usable hosts 2
CIDR prefix ip: 192.168.1.1/30

Hosts in network:
192.168.1.1
192.168.1.2


Simple script to find your devices name and IP address

In [1]:
import socket 
hostname = socket.gethostname() 
IPAddr = socket.gethostbyname(hostname) 
 
print("Your Computer Name is:" + hostname) 
print("Your Computer IP Address is:" + IPAddr) 

Your Computer Name is:DESKTOP-F7AAT91
Your Computer IP Address is:10.100.53.100


2. Dynamic Host Configuration Protocol (DHCP) 

a) DHCP Operation (4-Step Process) 
1. DHCP Discover (DHCPDISCOVER)  
o Client broadcasts a request for an IP address 
o Source: 0.0.0.0, Destination: 255.255.255.255 
o Contains client's MAC address for identification 
2. DHCP Offer (DHCPOFFER)  
o Server responds with an available IP address 
o Includes lease time and other configuration information 
o Multiple DHCP servers can respond with different offers 
3. DHCP Request (DHCPREQUEST)  
o Client broadcasts which offer it accepts 
o This informs all DHCP servers, allowing non-selected servers to 
reclaim offered IP addresses 
4. DHCP Acknowledgment (DHCPACK)  
o Selected server confirms the assignment 
o Finalizes lease duration and provides complete configuration 


b) Simulate DHCP operations 

Simple script simulate the process of DHCP

In [2]:

# dhcp_simple.py - Simplified DHCP Simulator 
 
# Server Configuration 
server = { 
    "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"], 
    "leases": {} 
} 
 
# Client Configuration 
client = { 
    "mac": "AA:BB:CC:DD:EE:FF", 
    "ip": None 
} 
 
def send_discover(): 
    print("\n[CLIENT] Step 1: Sending DHCP DISCOVER") 
    return { 
        "type": "DISCOVER", 
        "mac": client["mac"] 
    } 
 
def make_offer(discover): 
    print("\n[SERVER] Step 2: Making DHCP OFFER") 
    if not server["ip_pool"]: 
        print("No IPs available!") 
        return None 
     
    offered_ip = server["ip_pool"].pop(0) 
    return { 
        "type": "OFFER", 
        "mac": discover["mac"], 
        "ip": offered_ip 
    } 
 
def send_request(offer): 
    print("\n[CLIENT] Step 3: Sending DHCP REQUEST") 
    return { 
        "type": "REQUEST", 
        "mac": offer["mac"], 
        "ip": offer["ip"] 
    } 

def send_ack(request): 
    print("\n[SERVER] Step 4: Sending DHCP ACK") 
    server["leases"][request["mac"]] = request["ip"] 
    return { 
        "type": "ACK", 
        "mac": request["mac"], 
        "ip": request["ip"] 
    } 
 
def main(): 
    print("=== Simple DHCP Simulation ===") 
     
    # Client starts process 
    discover = send_discover() 
     
    # Server responds 
    offer = make_offer(discover) 
    if not offer: 
        return 
     
    # Client continues 
    request = send_request(offer) 
     
    # Server finalizes 
    ack = send_ack(request) 
     
    # Update client IP 
    client["ip"] = ack["ip"] 
     
    print("\n=== Result ===") 
    print(f"Client {client['mac']} got IP: {client['ip']}") 
    print("Server leases:", server["leases"]) 
 
if __name__ == "__main__": 
    main() 

=== Simple DHCP Simulation ===

[CLIENT] Step 1: Sending DHCP DISCOVER

[SERVER] Step 2: Making DHCP OFFER

[CLIENT] Step 3: Sending DHCP REQUEST

[SERVER] Step 4: Sending DHCP ACK

=== Result ===
Client AA:BB:CC:DD:EE:FF got IP: 192.168.1.100
Server leases: {'AA:BB:CC:DD:EE:FF': '192.168.1.100'}


Exercise 5: use the client sever example from previous week to have a TCP server as DHCP server and create a client that would get the IP address from the server following the process above.